# Featurestore - Mars21
## Feature Store Setup
Christos Aniftos \
Soeren Petersen

## Install required libraries

## Import modules

In [14]:
from google.cloud import aiplatform

In [15]:
from google.api_core import operations_v1
from google.cloud.aiplatform_v1beta1.types import io as io_pb2
from google.cloud.aiplatform_v1beta1.types import FeaturestoreMonitoringConfig
from google.cloud.aiplatform_v1beta1.types.feature import Feature
from google.cloud.aiplatform_v1beta1 import FeaturestoreServiceClient
from google.cloud.aiplatform_v1beta1 import FeaturestoreOnlineServingServiceClient
from google.cloud.aiplatform_v1beta1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1beta1.types import feature_selector as feature_selector_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_online_service as featurestore_online_service_pb2

In [4]:
PROJECT_ID = "feature-store-mars21" # Change to your project id
LOCATION = "us-central1" 
API_ENDPOINT = LOCATION+"-aiplatform.googleapis.com" 
FEATURESTORE_ID = "universe"
ENTITY="customer"

## Define clients for FS admin and data management

In [5]:
# Create admin_client for CRUD
admin_client = FeaturestoreServiceClient(
    client_options={"api_endpoint": API_ENDPOINT})

In [6]:
LOC_PATH = admin_client.common_location_path(PROJECT_ID, LOCATION)
FS_PATH = admin_client.featurestore_path(PROJECT_ID, LOCATION, FEATURESTORE_ID)
ENTITY_PATH = admin_client.entity_type_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY)
FEATURE_PATH = admin_client.feature_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY, '{}')

print("Location: \t", LOC_PATH)
print("Feature Store: \t", FS_PATH)
print("Entity: \t", ENTITY_PATH)
print("Feature: \t",FEATURE_PATH)

Location: 	 projects/feature-store-mars21/locations/us-central1
Feature Store: 	 projects/feature-store-mars21/locations/us-central1/featurestores/universe
Entity: 	 projects/feature-store-mars21/locations/us-central1/featurestores/universe/entityTypes/customer
Feature: 	 projects/feature-store-mars21/locations/us-central1/featurestores/universe/entityTypes/customer/features/{}


In [14]:
# Create operation client to poll LRO status.
#lro_client = operations_v1.OperationsClient(admin_client.transport.grpc_channel)

## Create FeatureStore

In [15]:
new_feature_store = featurestore_pb2.Featurestore(
            online_serving_config=featurestore_pb2.Featurestore.OnlineServingConfig(
                fixed_node_count=1 # The number of nodes for each cluster (bigtable)
            )
        )
try:
    print(
        admin_client.create_featurestore(
            featurestore_service_pb2.CreateFeaturestoreRequest(
                parent=LOC_PATH,
                featurestore_id=FEATURESTORE_ID,
                featurestore= new_feature_store
            )
        ).result()
    )
except Exception as ex:
    print(ex)

name: "projects/202835066335/locations/us-central1/featurestores/universe"

 ## List all Feature Stores

In [7]:
list(admin_client.list_featurestores(parent=LOC_PATH))

[name: "projects/202835066335/locations/us-central1/featurestores/universe"
 create_time {
   seconds: 1629202045
   nanos: 67779000
 }
 update_time {
   seconds: 1629416502
   nanos: 685422000
 }
 etag: "AMEw9yPBZRLTYXs6mTkVmurAuj_1jWOPJYh773b3w2bPGqTwiIElMRiNyThugfo6-qXk"
 online_serving_config {
   fixed_node_count: 1
 }
 state: STABLE]

## Get Feature store by name

In [8]:
try:
    feature_store_obj = admin_client.get_featurestore(name = FS_PATH)
except Exception as ex:
    print(ex)
feature_store_obj

name: "projects/202835066335/locations/us-central1/featurestores/universe"
create_time {
  seconds: 1629202045
  nanos: 67779000
}
update_time {
  seconds: 1629416502
  nanos: 685422000
}
etag: "AMEw9yPSjuqLfFPC0E0Gsuq850Dy89Bj0MZfmsbF3gmfx2tNAuQjJF7rJYvtUOZYk2u7"
online_serving_config {
  fixed_node_count: 1
}
state: STABLE

### Update feature store (changing online serving nodes)

In [9]:
feature_store_obj.online_serving_config.fixed_node_count=1
admin_client.update_featurestore(
    featurestore_service_pb2.UpdateFeaturestoreRequest(featurestore=feature_store_obj)
)

## Create a feature store Entity

In [10]:
try:
    entity_type_obj = admin_client.create_entity_type(
        featurestore_service_pb2.CreateEntityTypeRequest(
            parent=FS_PATH,
            entity_type_id=ENTITY, 
            entity_type=entity_type_pb2.EntityType(description="customer features"))).result()
    print(entity_type_obj)
except Exception as ex:
    print(ex)

name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/customer"
etag: "AMEw9yOknVfCmMFgVv-cmAPsOZwpypS5M-utDLiPIBB0Ypx4Pyc6"



In [11]:
type(entity_type_obj)

google.cloud.aiplatform_v1beta1.types.entity_type.EntityType

In [12]:
from google.protobuf import duration_pb2 as duration 

duration = duration.Duration()
duration.FromSeconds(seconds=60)


fmc=FeaturestoreMonitoringConfig(
    snapshot_analysis = FeaturestoreMonitoringConfig.SnapshotAnalysis(disabled=False, monitoring_interval = duration))
fmc

snapshot_analysis {
  monitoring_interval {
    seconds: 60
  }
}

In [13]:
entity_type_obj.monitoring_config= fmc
entity_type_obj = admin_client.update_entity_type(featurestore_service_pb2.UpdateEntityTypeRequest(
            entity_type=entity_type_pb2.EntityType(name=entity_type_obj.name)),
            )

## Create features in batch

In [23]:
features_list_tmp = []

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="gender",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="gender"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.BOOL,
                    description="weather customer is senior citizen ",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="SeniorCitizen"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.BOOL,
                    description="does customer have Partner",
                    monitoring_config= fmc,
                    labels=[("pii", ""),("approved", "")]),
                feature_id="Partner"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.BOOL,
                    description="does customer have Dependents",
                    monitoring_config= fmc,
                    labels=[("pii", ""),("approved", "")]),
                feature_id="Dependents"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.INT64,
                    description="how long he is a custormer for",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="tenure"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.BOOL,
                    description="is using PhoneService",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="PhoneService"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if customer has MultipleLines",
                    monitoring_config= fmc,
                    labels=[("no-pii", "")]),
                feature_id="MultipleLines"))


features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if customer has InternetService",
                    monitoring_config= fmc,
                    labels=[("no-pii", "")]),
                feature_id="InternetService"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if customer has OnlineSecurity",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="OnlineSecurity"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if customer has OnlineBackup",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="OnlineBackup"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if has device protection",
                    monitoring_config= fmc,
                    labels=[("no-pii", "")]),
                feature_id="DeviceProtection"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if uses tech support",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="TechSupport"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if has steaming tv service",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", ""),("validated", "")]),
                feature_id="StreamingTV"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if has steaming movies",
                    labels=[("no-pii", "")]),
                feature_id="StreamingMovies"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="type of contract",
                    labels=[("no-pii", "")]),
                feature_id="Contract"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.BOOL,
                    description="Opt in for PaperlessBilling",
                    labels=[("no-pii", "")]),
                feature_id="PaperlessBilling"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.STRING,
                    description="what is the payment method",
                    labels=[("no-pii", "")]),
                feature_id="PaymentMethod"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.DOUBLE_ARRAY,
                    description="average monthly charges",
                    labels=[("no-pii", "")]),
                feature_id="MonthlyCharges"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.BOOL,
                    description="if male",
                    labels=[("no-pii", "")]),
                feature_id="male"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.BOOL,
                    description="if female",
                    labels=[("no-pii", "")]),
                feature_id="female"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.DOUBLE,
                    description="Monthly chrages Standartised -1 - 1",
                    labels=[("no-pii", "")]),
                feature_id="MonthlyCharges_std"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.DOUBLE,
                    description="Total Charges",
                    labels=[("no-pii", "")]),
                feature_id="TotalCharges"))


features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.DOUBLE,
                    description="the average monthly spending of customer with similar profile",
                    labels=[("no-pii", "")]),
                feature_id="MonthlyCharges_avg_customer_clustered"))

try:
    features_list = admin_client.batch_create_features(
        parent=ENTITY_PATH,
        requests=features_list_tmp).result()
    print(features_list)
except Exception as ex:
    print(ex)

features {
  name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/planets/features/ls"
  etag: "AMEw9yM0tXiW24AnP1LBYf81wptyYS9ksETfNZr7WCJ162mRLFlU"
}
features {
  name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/planets/features/month"
  etag: "AMEw9yMucEqHsDqNcR7V6DSHQyJXHVphQCrfJqkcUFTAY2kbe9hS"
}
features {
  name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/planets/features/max_temp"
  etag: "AMEw9yMLrEmQIX-Z-bZJMPBicdWbvV9D-Xe6lFKyTbbLV4jzCwfM"
}
features {
  name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/planets/features/min_temp"
  etag: "AMEw9yNGTJ0UigaSuha9bVjYHL5VURDQNWhijg6RpxWyGoSWQQrS"
}
features {
  name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/planets/features/pressure"
  etag: "AMEw9yNBvkvW6Ss8sUXg61VF8KM_2kjkVd__SoFhwwbXSxfIAaJW"
}
features {
  name: "projects/202835066335/locations/us

## Update Feature

In [24]:
feature =admin_client.get_feature(name=FEATURE_PATH.format('MonthlyCharges'))
feature

name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/planets/features/atmo_opacity"
description: "opacity in atmosphere (i.e sunny)"
value_type: STRING
create_time {
  seconds: 1629202061
  nanos: 857196000
}
update_time {
  seconds: 1629202061
  nanos: 857196000
}
labels {
  key: "no-pii"
  value: ""
}
etag: "AMEw9yOBVsC1OSbtOzks3GC3YsMpG9SGIqJ0y2yX0-xRD1ScDX4HW-7ImeaFzVsjOT7H"
monitoring_config {
  snapshot_analysis {
    monitoring_interval {
      seconds: 86400
    }
    monitoring_interval_days: 1
  }
}

In [25]:
tmp_list = list(feature.labels.items())
tmp_list.append(("approved", ""))
feature.labels=tmp_list
feature.monitoring_config= fmc

In [26]:
admin_client.update_feature(
    featurestore_service_pb2.UpdateFeatureRequest(feature=feature)
)

name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/planets/features/atmo_opacity"
description: "opacity in atmosphere (i.e sunny)"
value_type: STRING
create_time {
  seconds: 1629202061
  nanos: 857196000
}
update_time {
  seconds: 1629202063
  nanos: 287767000
}
labels {
  key: "approved"
  value: ""
}
labels {
  key: "no-pii"
  value: ""
}
etag: "AMEw9yNlh4DkYxdNFpeJvKZyoDXQEn8PiDz-a2GogHC6EQZAa67TIUv5qxv8IgiBJqJA"
monitoring_config {
  snapshot_analysis {
    monitoring_interval {
      seconds: 86400
    }
    monitoring_interval_days: 1
  }
}

## Delete Feature

In [27]:
#try:
#    admin_client.delete_feature(name=FEATURE_PATH.format('last_five_max_temp')).result()
#except Exception as ex:
#    print(ex)

## Search Features

-  ``feature_id``: Supports = comparisons.
-  ``description``: Supports = comparisons. Multi-token filters should be enclosed in quotes.
-  ``entity_type_id``: Supports = comparisons.
-  ``value_type``: Supports = and != comparisons.
-  ``labels``: Supports key-value equality as well as key presence.
-  ``featurestore_id``: Supports = comparisons.

In [28]:
admin_client.search_features(location=LOC_PATH)

SearchFeaturesPager<features {
  name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/planets/features/arr_max_temp_3d"
  description: "array containing last 3 max temperatures"
  create_time {
    seconds: 1629202061
    nanos: 875339000
  }
  update_time {
    seconds: 1629202061
    nanos: 875339000
  }
  labels {
    key: "no-pii"
    value: ""
  }
}
features {
  name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/planets/features/arr_min_temp_3d"
  description: "array containing last 3 min temperatures"
  create_time {
    seconds: 1629202061
    nanos: 873703000
  }
  update_time {
    seconds: 1629202061
    nanos: 873703000
  }
  labels {
    key: "no-pii"
    value: ""
  }
}
features {
  name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/planets/features/arr_pressure_3d"
  description: "array containing last 3 atmospheric pressures"
  create_time {
    seconds: 1629202061
    

In [17]:
q="featurestore_id={}".format(FEATURESTORE_ID)
admin_client.search_features(
        featurestore_service_pb2.SearchFeaturesRequest(
            location=LOC_PATH, query=q, page_size=2, page_token="" ))

SearchFeaturesPager<features {
  name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/planets/features/arr_max_temp_3d"
  description: "array containing last 3 max temperatures"
  create_time {
    seconds: 1628001829
    nanos: 658224000
  }
  update_time {
    seconds: 1628001829
    nanos: 658224000
  }
  labels {
    key: "no-pii"
    value: ""
  }
}
features {
  name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/planets/features/arr_min_temp_3d"
  description: "array containing last 3 min temperatures"
  create_time {
    seconds: 1628001829
    nanos: 656134000
  }
  update_time {
    seconds: 1628001829
    nanos: 656134000
  }
  labels {
    key: "no-pii"
    value: ""
  }
}
next_page_token: "CoYBCICAgICAgJTnOhJ4QU1Fdzl5TzNfbV8zQm5JVjljeU1kSl9YbFh5c2VKYjNiMEJHT2pocTZqdjAyVGFOMXdSQ212Z2x1bDZFczdDci0weUFUbF81c245eU9wQWpQU0RWcW9hNHpYVDNWWXYzc2U4MC02R1hvQ2MwNlJtZkl2OURPZz09GAI="
>

In [18]:
q="""
    feature_id=MonthlyCharges AND 
    value_type=DOUBLE AND 
    featurestore_id={}
""".format(FEATURESTORE_ID)

admin_client.search_features(
        featurestore_service_pb2.SearchFeaturesRequest(
            location=LOC_PATH, query=q, page_size=5))

SearchFeaturesPager<features {
  name: "projects/202835066335/locations/us-central1/featurestores/universe/entityTypes/planets/features/arr_max_temp_3d"
  description: "array containing last 3 max temperatures"
  create_time {
    seconds: 1628001829
    nanos: 658224000
  }
  update_time {
    seconds: 1628001829
    nanos: 658224000
  }
  labels {
    key: "no-pii"
    value: ""
  }
}
>